In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"         # "microsoft/orca-math-word-problems-200k"

# Fine-tuned model name
new_model = "Llama-2-7b-chat-finetune"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "/content/sample_data"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [26]:
# from langchain_community.document_loaders import PyPDFLoader

In [27]:
# pdf_load = PyPDFLoader("/content/sample_data/Chatbot Data/MACHINE LEARNING(R17A0534).pdf").load()

In [28]:
# import re
# for i in range(len(pdf_load)):
#   text = pdf_load[i].page_content
#   clean_text = re.sub(r'[^\x00-\x7F]+', '', text)
#   clean_text_with_spaces = clean_text.replace("\n", "")
#   pdf_load[i].page_content = clean_text_with_spaces

In [29]:
# pdf_load[0].page_content

In [30]:
# pdf_load

In [31]:
# import PyPDF2

# def extract_text_from_pdf(pdf_path):
#     text = ""
#     with open(pdf_path, 'rb') as file:
#         reader = PyPDF2.PdfReader(file)
#         for page_num in range(len(reader.pages)):
#             page = reader.pages[page_num]
#             text += page.extract_text()
#     return text

# pdf_text = extract_text_from_pdf('/content/sample_data/Chatbot Data/MACHINE LEARNING(R17A0534).pdf')



In [32]:
# print(pdf_text)

In [33]:
# cc

In [34]:
# import pytesseract
# from PIL import Image

# # Load PDF and convert each page to an image
# # Use a PDF library like PyPDF2 or pdf2image for this step

# # Extract text from each image using Tesseract OCR
# extracted_text = []
# for page_image in pdf_load:
#     text = pytesseract.image_to_string(page_image)
#     extracted_text.append(text)

In [35]:
# Load dataset (you can process it here)
# dataset = load_dataset(dataset_name, split="train")

In [3]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit, # True
    bnb_4bit_quant_type=bnb_4bit_quant_type,  # float16
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,)  # False

In [4]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [5]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = device_map
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model.config.use_cache = False
model.config.pretraining_tp = 1

In [7]:
# Load LLaMA Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [8]:
# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    r = lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [9]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [43]:
# from transformers import DataCollatorWithPadding
# pdf_text = DataCollatorWithPadding(tokenizer=tokenizer)

In [44]:
# pdf_text

In [10]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name)

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})

In [12]:
train_dataset = dataset["train"]

In [13]:
split_dataset = train_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 900
    })
    test: Dataset({
        features: ['text'],
        num_rows: 100
    })
})


In [14]:
data = split_dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)

In [15]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 900
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})

In [16]:
data["train"]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 900
})

In [53]:
# def formating_func(example):
#   text = f"Question: {example['question'][0]} \nAnswer: {example['answer'][0]}"
#   return [text]

In [54]:
# Set supervised fine-tuning parameters
#dataset_text_field="answer",
# max_seq_length=max_seq_length,
# tokenizer=tokenizer,
# packing=False,



# trainer = SFTTrainer(
#     model=model,
#     train_dataset=data["train"],
#     peft_config=peft_config,
#     args=training_arguments,
#     formatting_func = formating_func,
# )

In [55]:
# # Train model
# trainer.train()

In [56]:
# # Ignore warnings
# logging.set_verbosity(logging.CRITICAL)

# # Run text generation pipeline with our next model
# prompt = "A number divided by 10 is 6. Yoongi got the result by subtracting 15 from a certain number. What is the result he got?"
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
# results = pipe(f"{prompt}")

In [57]:
# if len(dataset) == 0:
#     print("Dataset is empty!")
# else:
#     # Split the dataset into training and testing sets
#     train_data, test_data = train_test_split(dataset["train"], test_size=0.2, random_state=42)

In [17]:
data["test"]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 100
})

In [59]:
# from sklearn.model_selection import train_test_split

In [60]:
# train_data, test_data = train_test_split(dataset["train"], test_size=0.2, random_state=42)

In [61]:
# train_data

In [62]:
# train_data["answer"]

In [63]:
# # Splitting the dataset into training and testing sets
# train_data, test_data = train_test_split(dataset['train'], test_size=0.2, random_state=42)

In [64]:
# from datasets import Dataset, DatasetDict

# # Apna dataset define karein, yeh sirf ek example hai
# train_dataset = Dataset({
#     'features': train_data
# })

# # Dataset ko DatasetDict mein daalein
# dataset_dict = DatasetDict({'train': train_dataset})


In [65]:
# dataset_dict

In [66]:
# train_data = train_data.map(lambda samples: tokenizer(samples["question"]), batched=True)

In [67]:
# def formating_func(example):
#   text = f"Question: {example['question'][0]} \nAnswer: {example['answer'][0]}"
#   return [text]

In [18]:
trainer_2 = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    max_seq_length=max_seq_length,
    packing=False,
)

# Set supervised fine-tuning parameters
#dataset_text_field="answer",
# max_seq_length=max_seq_length,
# tokenizer=tokenizer,
# packing=False,

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [71]:
# training_arguments

In [19]:
# Train model
trainer_2.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.504300
50,1.762500
75,1.140700
100,1.509900
125,1.138100
150,1.476600
175,1.178900
200,1.500700
225,1.400800


TrainOutput(global_step=225, training_loss=1.4013831583658853, metrics={'train_runtime': 1333.8545, 'train_samples_per_second': 0.675, 'train_steps_per_second': 0.169, 'total_flos': 7864935095009280.0, 'train_loss': 1.4013831583658853, 'epoch': 1.0})

In [20]:
save_dir = '/content/sample_data/Model Save/'

trainer_2.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: /content/sample_data/Model Save/
